In [10]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import json
import pytz 
from IPython.display import JSON
from keys import RapidAPI_Key
from keys import MySQL_Pass

def create_sql_connection():
    schema = "sql_escooter_forecast"
    host = "127.0.0.1"
    user = "root"
    password = MySQL_Pass
    port = 3306
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    return connection_string

def read_airports_from_sql(connection_str):
    airports_df = pd.read_sql('SELECT city_id, icao FROM airports', connection_str)
    return airports_df


#def get_airports_in_city(cities_df): #this function is not used right now. The icao code for Berlin is hardcoded in function call
#    city_name = cities_df['city']
#    url = "https://aerodatabox.p.rapidapi.com/airports/search/location"
#    querystring = {"lat":"40.688812","lon":"-74.044369","radiusKm":"50","limit":"10","withFlightInfoOnly":"true"}
#    headers = {
#        "X-RapidAPI-Key": "SIGN-UP-FOR-KEY",
#        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
#    }
#    response = requests.get(url, headers=headers, params=querystring)
#    print(response.json())
#   
#    return icao_list
    
def retrieve_flight_data(airports_df):
    flight_items = []
    berlin_timezone = pytz.timezone("Europe/Berlin")  # Define Berlin timezone
    current_datetime = datetime.now()
    next_day_datetime = current_datetime + timedelta(days=1)
    
    icao_list = airports_df['icao'].tolist()
    
    for icao in icao_list:
        # Define the time slots
        time_slots = [
            (next_day_datetime.replace(hour=0, minute=0, second=0, microsecond=0),
             next_day_datetime.replace(hour=11, minute=59, second=59, microsecond=999999)),
            (next_day_datetime.replace(hour=12, minute=0, second=0, microsecond=0),
             next_day_datetime.replace(hour=23, minute=59, second=59, microsecond=999999))
        ]

        retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
        url_base = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/"
        
        headers = {
            "X-RapidAPI-Key": RapidAPI_Key,
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }

        for i, (start_time, end_time) in enumerate(time_slots, start=1):
            # Format the dates in the required format for the API
            start_datetime = start_time.strftime("%Y-%m-%dT%H:%M")
            end_datetime = end_time.strftime("%Y-%m-%dT%H:%M")

            url = f"{url_base}{start_datetime}/{end_datetime}"

            querystring = {
                "withLeg": "true",
                "direction": "Arrival",
                "withCancelled": "false",
                "withCodeshared": "false",
                "withCargo": "false",
                "withPrivate": "false",
                "withLocation": "false",
                "start": start_datetime,
                "end": end_datetime
            }

            response = requests.get(url, headers=headers, params=querystring)

            # Extract JSON data from response
            data = response.json()

            for item in data["arrivals"]:
                flight_item = {
                    "arrival_airport_icao": icao,
                    "departure_airport_icao": item["departure"]["airport"].get("icao", None),
                    "departure_airport_name": item["departure"]["airport"].get("name", None),
                    "scheduled_arrival_time": item["arrival"]["scheduledTime"].get("local", None),
                    "flight_number": item.get("number", None),
                    "data_retrieval_time": retrieval_time
                }

                flight_items.append(flight_item)
                
            flights_df = pd.DataFrame(flight_items)
            #flights_df["scheduled_arrival_time"] = flights_df["scheduled_arrival_time"].str[:-6]
            flights_df['scheduled_arrival_time'] = pd.to_datetime(flights_df['scheduled_arrival_time']).dt.tz_localize(None)
            flights_df["scheduled_arrival_time"] = pd.to_datetime(flights_df["scheduled_arrival_time"])
            flights_df["data_retrieval_time"] = pd.to_datetime(flights_df["data_retrieval_time"])
    
    return flights_df

def push_flights_data_to_sql(flights_df, conn_str):
    
    flights_df.to_sql('flights',
                      if_exists='append',
                      con=conn_str,
                      index=False)
    return 'Done Successfully!'

In [11]:
con_str = create_sql_connection()
airports_df = read_airports_from_sql(con_str)
#icao_list = get_airports_in_city(city_df)
#icao_list = ["EDDB"]  
flight_data = retrieve_flight_data(airports_df)
push_flights_data_to_sql(flight_data, con_str)

'Done Successfully!'

In [14]:
airports_df

,city_id,icao
0,1,EDDB
1,2,EDDM
